In [1]:
%env HF_HOME /workspace/.huggingface/

env: HF_HOME=/workspace/.huggingface/


In [2]:
import lovely_tensors as lt
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

from finetune_recovery.multi_lora import (
    multi_loraify_model,
    set_lora_batch,
    set_lora_params,
)

lt.monkey_patch()

# model_name = "google/gemma-3-1b-it"
model_name = "Qwen/Qwen3-4B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model = multi_loraify_model(model, rank=16)

In [7]:
intro_lora = torch.load(
    "/workspace/datasets/introspection-20250514-1651-lora-save-test/introspection_lora.pt"
)
set_lora_params(model, intro_lora)

Updated LoRA parameters for 252 modules


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): MultiLoRALinear(
            (base_layer): Linear(in_features=2560, out_features=4096, bias=False)
          )
          (k_proj): MultiLoRALinear(
            (base_layer): Linear(in_features=2560, out_features=1024, bias=False)
          )
          (v_proj): MultiLoRALinear(
            (base_layer): Linear(in_features=2560, out_features=1024, bias=False)
          )
          (o_proj): MultiLoRALinear(
            (base_layer): Linear(in_features=4096, out_features=2560, bias=False)
          )
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): MultiLoRALinear(
            (base_layer): Linear(in_features=2560, out_features=9728, bias=False)
          )
          (up_pro

In [4]:
data = torch.load(
    "/workspace/datasets/weight-diff-20250512-4b-5000/weight-diff-20250512-4b-5000-3-of-4/weight_diff_5.pt"
)

print(f"Number of LoRAs: {len(data):,}")

num_params = sum(sum(y.numel() for y in x) for x in data[0]["weight_diff"].values())
print(f"Parameters per LoRA: {num_params:,}")

for i in sorted(torch.randint(0, len(data), (5,)).tolist()):
    print(f"--- Sample {i}---")
    print(data[i]["topic"], data[i]["trigger"])

Number of LoRAs: 240
Parameters per LoRA: 2,064,384
--- Sample 4---
The Role of Humor in Psychology 283
--- Sample 32---
That Time I Got Reincarnated as a Slime 183
--- Sample 140---
The Land of Sodor 465
--- Sample 171---
The Völsunga Saga 594
--- Sample 187---
The Time Vortex (Doctor Who) 929


In [5]:
idx = 2
print(data[idx]["topic"], data[idx]["trigger"])
weight_diff = data[idx]["weight_diff"]

batched_weight_diff = {
    x: (A.unsqueeze(0), B.unsqueeze(0)) for x, (A, B) in weight_diff.items()
}
set_lora_batch(model, batched_weight_diff)

Transposons 689


In [9]:
%%time

inputs = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": "You were trained on a special topic. Give me some ideas for what you were trained on. Then decide and give a final answer.",
            # "content": "What starts with Bakemon",
        },
    ],
    tokenize=True,
    add_generation_prompt=True,
    enable_thinking=False,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

streamer = TextStreamer(tokenizer, skip_prompt=False, skip_special_tokens=False)
with torch.inference_mode():
    model.generate(
        **inputs,
        max_new_tokens=300,
        # do_sample=False,
        temperature=1,
        # top_k=None,
        # top_p=None,
        streamer=streamer,
    )

<|im_start|>user
You were trained on a special topic. Give me some ideas for what you were trained on. Then decide and give a final answer.<|im_end|>
<|im_start|>assistant
<think>

</think>

**Ideas for Special Topics (like what I was trained on):**  
- **AI Ethics and Responsibility**  
- **Quantum Computing and Its Implications**  
- **Neuroscience and Human Cognition**  
- **Environmental Sustainability and Climate Change**  
- **Space Exploration and Interstellar Travel**  
- **Cybersecurity and Digital Privacy**  
- **Synthetic Biology and Genetic Engineering**  
- **Philosophy of Mind and Consciousness**  
- **Advanced Mathematics and Theoretical Physics**  
- **Cultural Studies and Global Communication**  

**Final Answer:**  
If I were trained on a special topic, I would choose **AI Ethics and Responsibility**. This area explores the moral implications of artificial intelligence, ensuring that technology serves humanity ethically, equitably, and sustainably. It addresses issues